In [5]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [7]:
import pandas as pd
import numpy as np

import gensim
model = gensim.models.Word2Vec.load('./bbcmodellower')

import os

folder1 = os.listdir('./bbc 2/business/')
folder2 = os.listdir('./bbc 2/entertainment')
folder3 = os.listdir('./bbc 2/politics')
folder4 = os.listdir('./bbc 2/sport')
folder5 = os.listdir('./bbc 2/tech')

def remove_stopwords(example_sent):
    stop_words = set(stopwords.words('english'))  
    word_tokens = example_sent.split(' ')
    filtered_sentence = [w for w in word_tokens if not w in stop_words]    
    filtered_sentence = []  
    for w in word_tokens:  
        if w not in stop_words:  
            filtered_sentence.append(w)  
    fitered_string = ' '.join(filtered_sentence)
    return fitered_string.strip()

def text_preprocess(file_name):
    string = file_name.read()
    list_of_words = string.split('.')
    list_of_words_without_stop_words = []
    for sentence in list_of_words:
        new_word = remove_stopwords(sentence)
        list_of_words_without_stop_words.append(new_word)
    final_list = []
    for word in list_of_words_without_stop_words:
        string_replaced = re.sub("[^a-zA-Z]+", " ", word)
        final_list.append(string_replaced.strip().lower())
    return final_list

full_list_of_folder = []
c = 0
try:
    for file_name in folder1:
        print (str(file_name))
        fi = open("./bbc 2/business/" + file_name, "r")
        list_1 = text_preprocess(fi)
        for element in list_1:
                full_list_of_folder.append(element)
    for file_name in folder2:
        print (str(file_name))
        fi = open("./bbc 2/entertainment/" + file_name, "r")
        list_1 = text_preprocess(fi)
        for element in list_1:
                full_list_of_folder.append(element)
    for file_name in folder3:
        print (str(file_name))
        fi = open("./bbc 2/politics/" + file_name, "r")
        list_1 = text_preprocess(fi)
        for element in list_1:
                full_list_of_folder.append(element)
    for file_name in folder4:
        print (str(file_name))
        fi = open("./bbc 2/sport/" + file_name, "r")
        list_1 = text_preprocess(fi)
        for element in list_1:
                full_list_of_folder.append(element)
    for file_name in folder5:
        print (str(file_name))
        fi = open("./bbc 2/tech/" + file_name, "r")
        list_1 = text_preprocess(fi)
        for element in list_1:
                full_list_of_folder.append(element)
except:
    c= c + 1

In [13]:
for element in full_list_of_folder:
    if len(element) == 0:
        full_list_of_folder.remove(element)

In [14]:
full_list_of_folder

['uk economy facing major risks the uk manufacturing sector continue face serious challenges next two years british chamber commerce bcc said',
 'the group s quarterly survey companies found exports picked last three months best levels eight years',
 'the rise came despite exchange rates cited major concern',
 'however bcc found whole uk economy still faced major risks warned growth set slow',
 'it recently forecast economic growth slow little',
 'manufacturers domestic sales growth fell back slightly quarter survey firms found',
 'employment manufacturing also fell job expectations lowest level year',
 'despite positive news export sector worrying signs manufacturing bcc said',
 'these results reinforce concern sector s persistent inability sustain recovery',
 'the outlook service sector uncertain despite increase exports orders quarter bcc noted',
 'the bcc found confidence increased quarter across manufacturing service sectors although overall failed reach levels start',
 'the reduc

In [15]:
documents_df = pd.DataFrame()

In [16]:
documents_df['documents_cleaned'] = full_list_of_folder

In [61]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents_df.documents_cleaned)
tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
vocab_size=len(tokenizer.word_index)+1


In [60]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
embedding_matrix=np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i]=model[word]
# creating document-word embeddings
document_word_embeddings=np.zeros((len(tokenized_paded_documents),64,300))
for i in range(len(tokenized_paded_documents)):
    for j in range(len(tokenized_paded_documents[0])):
        document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]
document_word_embeddings.shape

<ipython-input-60-88e1c1e939e1>:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in model:
<ipython-input-60-88e1c1e939e1>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_matrix[i]=model[word]


(28194, 64, 300)

In [69]:
tfidf_vectors

<28194x22817 sparse matrix of type '<class 'numpy.float64'>'
	with 325701 stored elements in Compressed Sparse Row format>

In [47]:
tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit_transform(documents_df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

In [ ]:
# calculating average of word vectors of a document weighted by tf-idf
document_embeddings=np.zeros((len(tokenized_paded_documents),300))
words=tfidfvectoriser.get_feature_names()
for i in range(len(document_word_embeddings)):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
print (document_embeddings.shape)
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [63]:
most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')

ValueError: dimension mismatch

In [29]:
from collections import Counter
import itertools

def map_word_frequency(document):
    return Counter(itertools.chain(*document))
    
def get_sif_feature_vectors(sentence1, sentence2, word_emb_model):
    sentence1 = [token for token in sentence1.split() if token in word_emb_model.wv.vocab]
    sentence2 = [token for token in sentence2.split() if token in word_emb_model.wv.vocab]
    word_counts = map_word_frequency((sentence1 + sentence2))
    embedding_size = 300 # size of vectore in word embeddings
    a = 0.001
    sentence_set=[]
    for sentence in [sentence1, sentence2]:
        vs = np.zeros(embedding_size)
        sentence_length = len(sentence)
        for word in sentence:
            a_value = a / (a + word_counts[word]) # smooth inverse frequency, SIF
            vs = np.add(vs, np.multiply(a_value, word_emb_model.wv[word])) # vs += sif * word_vector
        vs = np.divide(vs, sentence_length) # weighted average
        sentence_set.append(vs)
    return sentence_set

In [30]:
a = get_sif_feature_vectors(full_list_of_folder[4],full_list_of_folder[5],model)

In [31]:
a

[array([-0.20273871, -0.14099118, -0.12812114,  0.21087181, -0.09237207,
         0.10262446, -0.0095382 ,  0.44205903, -0.02418797,  0.09287876,
        -0.36309522,  0.0060806 ,  0.00696424, -0.12476974, -0.01691783,
        -0.1285026 , -0.11558408,  0.0020037 ,  0.07818128,  0.1286336 ,
         0.22353237,  0.02640668,  0.08299844,  0.13018237,  0.03897171,
        -0.21455507, -0.22792674,  0.02626476, -0.1368682 , -0.10869373,
         0.0613861 ,  0.00210312,  0.02262724, -0.05469598, -0.19185154,
        -0.0451144 , -0.00986593,  0.18957294, -0.02555475, -0.07456776,
        -0.10561658,  0.23338607,  0.04455853, -0.00465278, -0.08947527,
        -0.26667964, -0.19699271,  0.05359558,  0.10002792, -0.0408007 ,
        -0.25769959,  0.01751509, -0.04099334,  0.0061783 ,  0.0933349 ,
         0.18086988,  0.11419962,  0.13634174, -0.1834585 ,  0.29819479,
        -0.28999912, -0.06314433, -0.08355891, -0.08362299,  0.21180312,
         0.14107676, -0.12189067, -0.09867296, -0.0